In [2]:
import os
import cv2
import numpy as np 

# Specify the path to the directory containing your images
main_directory_path = r'D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)'

# Get a list of all files in the directory
files = os.listdir(main_directory_path)

# Filter only files with certain extensions (e.g., .jpg, .jpeg, .png)
image_files = [file for file in files ]

# Iterate over the image files and read each image using OpenCV
pathlst = []
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(main_directory_path, image_file)

    # Use OpenCV to read the image data
    print(image_path)
    
    pathlst.append(image_path)

D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\001
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\002
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\003
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\004
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\005
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\006
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\007
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\008
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\009
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\010
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\011
D:\1-  (Graduatoin Project)\F4 Gait recognation\1- Data\train\Final data(69)\012
D:\1-  (Graduatoin Project)\

## Read silhouette

In [2]:
import cv2
import numpy as np
import os

# Function to read silhouette images from a directory
def read_silhouette_images(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img_path = os.path.join(directory_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Resize the image to a common size if needed
            img = cv2.resize(img, (width, height))

            images.append(img)
    return images









## covariance method


In [ ]:
def create_gei_covariance(silhouette_images, regularization=1e-5):
    # Assuming all silhouette images have the same dimensions
    height, width = silhouette_images[0].shape

    # Reshape the images into 1D arrays
    flattened_images = [img.flatten() for img in silhouette_images]

    # Stack the flattened images into a 2D array (rows are pixels, columns are images)
    stacked_images = np.stack(flattened_images, axis=1)

    # Compute the covariance matrix with regularization
    covariance_matrix = np.cov(stacked_images) + regularization * np.eye(stacked_images.shape[0])

    # Compute the eigenvectors of the covariance matrix
    _, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Use the eigenvector corresponding to the largest eigenvalue as the GEI
    gei = eigenvectors[:, -1].reshape(height, width)

    # Normalize the GEI to be in the range [0, 255]
    gei = ((gei - gei.min()) / (gei.max() - gei.min()) * 255).astype(np.uint8)

    return gei



## mean method (flip)

In [3]:
import numpy as np

def create_gei(silhouette_images, flip=True):
    # Assuming all silhouette images have the same dimensions
    height, width = silhouette_images[0].shape

    # Initialize an array to store accumulated pixel values
    gei = np.zeros((height, width), dtype=np.float32)

    # Accumulate pixel values across the sequence
    for img in silhouette_images:
        # Optionally flip the silhouette image
        if flip:
            img = np.fliplr(img)

        gei += img.astype(np.float32)

    # Normalize the accumulated values
    gei /= len(silhouette_images)

    # Convert the GEI to uint8 for display or further processing
    gei = gei.astype(np.uint8)

    return gei


## alternative method (flip)

In [4]:
import numpy as np
import cv2

def create_gei_alternative(silhouette_images, flip=True):
    # Assuming all silhouette images have the same dimensions
    height, width = silhouette_images[0].shape

    # Optionally flip the silhouette images horizontally
    if flip:
        silhouette_images = [np.fliplr(img) for img in silhouette_images]

    # Stack the silhouette images along the time dimension
    stacked_images = np.stack(silhouette_images, axis=0)

    # Compute the temporal average along the time dimension
    gei = np.mean(stacked_images, axis=0)

    # Normalize the GEI to values between 0 and 255
    gei = ((gei - np.min(gei)) / (np.max(gei) - np.min(gei))) * 255

    # Convert the GEI to uint8 for display or further processing
    gei = gei.astype(np.uint8)

    # Save the GEI image to a file if a filename is provided

    return gei


In [11]:
import numpy as np
import cv2

def standard_deviation(silhouette_images, flip=True):
    # Assuming all silhouette images have the same dimensions
    height, width = silhouette_images[0].shape

    # Optionally flip the silhouette images horizontally
    if flip:
        silhouette_images = [np.fliplr(img) for img in silhouette_images]

    # Stack the silhouette images along the time dimension
    stacked_images = np.stack(silhouette_images, axis=0)

    # Compute the temporal standard deviation along the time dimension
    std_dev = np.std(stacked_images, axis=0)

    # Normalize the standard deviation to values between 0 and 255
    std_dev_normalized = ((std_dev - np.min(std_dev)) / (np.max(std_dev) - np.min(std_dev))) * 255

    # Convert the normalized standard deviation to uint8 for display or further processing
    std_dev_image = std_dev_normalized.astype(np.uint8)

    # Save the standard deviation image to a file if needed

    return std_dev_image


## Augmented GEI

In [5]:
import numpy as np
import cv2

def create_augmented_gei(gei, num_additional=4, save_prefix='augmented_gei'):
    augmented_images = [gei]

    for i in range(num_additional):
        # Randomly rotate the GEI (angle in degrees)
        angle = np.random.uniform(-10, 10)
        rotated_gei = rotate_image(gei, angle)

        # Randomly scale the GEI
        scale_factor = np.random.uniform(0.9, 1.1)
        scaled_gei = scale_image(rotated_gei, scale_factor)

        augmented_images.append(scaled_gei)

    # # Save the augmented images
    # for i, img in enumerate(augmented_images):
    #     filename = f'{save_prefix}_{i}.png'
    #     cv2.imwrite(filename, img)

    return augmented_images

def rotate_image(image, angle):
    rows, cols = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    return rotated_image

def scale_image(image, scale_factor):
    rows, cols = image.shape
    scaled_image = cv2.resize(image, (int(cols * scale_factor), int(rows * scale_factor)), interpolation=cv2.INTER_LINEAR)
    return scaled_image




## code to preform 


In [3]:
##Specify the desired dimensions for silhouette images
height, width = 100, 50  # Replace with your desired dimensions

# Specify the path to the directory containing silhouette images
for i in pathlst:
    _, filename = os.path.split(i)
    print(filename)
    directory_path = f"D:\\1-  (Graduatoin Project)\\F4 Gait recognation\\1- Data\\real\\Amigos\\Amigos\\NEW_GEI_AMIGOS\\{filename}"
    os.makedirs(directory_path, exist_ok=True)
    pth= os.listdir(i)
    for j in pth :
        #print(j)
        path_img = os.path.join(i, j)
        #print(path_img)
        _, filename2 = os.path.split(j)
        #print(filename2)
        directory_path2 = f"D:\\1-  (Graduatoin Project)\\F4 Gait recognation\\1- Data\\real\\Amigos\\Amigos\\NEW_GEI_AMIGOS\\{filename}"
        #os.makedirs(directory_path2, exist_ok=True)
        image_files = [file for file in os.listdir(path_img) if file.endswith('.jpg') or file.endswith('.png')]
        #print(image_files)
        x=0
    
            
        silhouette_images = read_silhouette_images(path_img)
            #plt.imshow(estimated_image)
        gei_image = create_gei(silhouette_images,False)
        output_filename = f"{image_files[x]}"
        output_path = os.path.join(directory_path2, output_filename)
        cv2.imwrite(output_path,gei_image)
        x+=1

        image_files=[]
    


000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069


In [4]:
# ...
for i in pathlst:
    _, filename = os.path.split(i)
    print(filename)
    directory_path = f"D:\\1-  (Graduatoin Project)\\F4 Gait recognation\\1- Data\\real\\Amigos\\Amigos\\GEI_Amigos\\{filename}"
    pth= os.listdir(i)
    for j in pth:
        path_img = os.path.join(i, j)
        _, filename2 = os.path.split(j)
        directory_path2 = f"D:\\1-  (Graduatoin Project)\\F4 Gait recognation\\1- Data\\real\\Amigos\\Amigos\\GEI_Amigos\\{filename}"
        image_files = [file for file in os.listdir(path_img) if file.endswith('.jpg') or file.endswith('.png')]
        x = 0

        silhouette_images = read_silhouette_images(path_img)
        gei_image = standard_deviation(silhouette_images, False)
        aug_images = create_augmented_gei(gei_image, num_additional=10)

        for aug_img in aug_images:
            output_filename = f"aug_no_3_2{image_files[x]}"
            output_path = os.path.join(directory_path2, output_filename)
            cv2.imwrite(output_path, aug_img)
            x += 1

        image_files = []




000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
